In this practical work, we will be exploring some functionalities of the *MLflow model registry* like adding an MLflow Model to the Model Registry, transitioning it between different stages, etc

We will be doing the following steps:

1. Setting up data
2. Creating a training pipeline
3. Tracking the training metadata and artifcats using [*MLflow Tracking*](https://www.mlflow.org/docs/latest/tracking.html)
4. MLflow registry hands-on
5. Retraining pipeline

To go deeper on the *MLflow model registry*, check out [the official documentation](https://www.mlflow.org/docs/latest/model-registry.html)

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mlflow
from mlflow.tracking import MlflowClient

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

np.random.seed(42)

plt.rcParams["figure.figsize"] = [7, 5]
pd.set_option("display.max_columns", None)

# 1. Data setup

In [141]:
df = pd.read_csv("../data/power_plants.csv")
df.head()

,AT,V,AP,RH,PE
0,14.96,41.76,1024.07,73.17,463.26
1,25.18,62.96,1020.04,59.08,444.37
2,5.11,39.40,1012.16,92.14,488.56
3,20.86,57.32,1010.24,76.64,446.48
4,10.82,37.50,1009.23,96.62,473.90


We want to simulate having multiple models in production at the same time (one model per IoT device):
- Create a new column `device_id` with the IoT device id by assigning a random number (device_id) between 1 and 4:

In [142]:
import numpy as np

df["device_id"] = np.random.randint(1, 5, size=len(df))
df.head()

,AT,V,AP,RH,PE,device_id
0,14.96,41.76,1024.07,73.17,463.26,3
1,25.18,62.96,1020.04,59.08,444.37,4
2,5.11,39.40,1012.16,92.14,488.56,1
3,20.86,57.32,1010.24,76.64,446.48,3
4,10.82,37.50,1009.23,96.62,473.90,3


In [22]:
df.device_id.value_counts()

1    12081
4    11995
3    11897
2    11867
Name: device_id, dtype: int64

# 2. Model training

## 2.1. Train models for the different IoT devices

### Train model for a single IoT device:
Create a function that trains a model for a single device. It should take the device dataframe as input an do the following steps:
- Split the data into training and test set
- Trains a `RandomForestRegressor`
- Evaluates it with `MAE`, `MSE` and `RMSE`

In [5]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_model_for_an_iot_device(device_df: pd.DataFrame) -> None:
    """Trains a model for a single device"""
    device_id = device_df.iloc[0].device_id
    
    # Split data
    X = device_df[["AT", "V", "AP", "RH"]]
    y = device_df["PE"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
    # Fit model
    model = RandomForestRegressor()
    model.fit(X_train, y_train)

    # Evaluate the model
    y_pred = model.predict(X_test)
    
    test_mae = mean_absolute_error(y_test, y_pred)
    test_mse = mean_squared_error(y_test, y_pred)
    test_rmse = mean_squared_error(y_test, y_pred, squared=False)

### Train models for all the devices

Now that we can train a model for a given IoT device, let's orchestrate the training for all device models given the full dataset (sensor measures from all the IoT devices).

* Create a function `train_models_for_all_iot_devices` that will take the full data and call the `train_model_for_an_iot_device` function with the corresponding device data

In [6]:
def train_models_for_all_iot_devices(data: pd.DataFrame) -> None:
    for device_id in data.device_id.unique():
        iot_device_df = data[data.device_id == device_id]
        train_model_for_an_iot_device(iot_device_df)

In [7]:
train_models_for_all_iot_devices(df)

# 3. Tracking the training metadata and Artifacts using MLflow Tracking

`mlflow server --backend-store-uri sqlite:////tmp/mlruns.db --default-artifact-root /tmp/mlruns`

In [149]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [150]:
experiment_name = "Iot device model training"

mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='/tmp/mlruns/1', creation_time=1671180459441, experiment_id='1', last_update_time=1671180459441, lifecycle_stage='active', name='Iot device model training', tags={}>

In [10]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_model_for_an_iot_device(device_df: pd.DataFrame) -> None:
    """ Trains a model for a single device """
    device_id = device_df.iloc[0].device_id
    
    with mlflow.start_run():
        mlflow.sklearn.autolog()

        # Split data
        X = device_df[["AT", "V", "AP", "RH"]]
        y = device_df["PE"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        mlflow.log_params(
            {"nb_sample": len(X), "nb_training_samples": len(X_train), "nb_testing_samples": len(X_test)})

        # Fit model
        model = RandomForestRegressor()
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)

        test_mae = mean_absolute_error(y_test, y_pred)
        test_mse = mean_squared_error(y_test, y_pred)
        test_rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metrics({"test_mae": test_mae, "test_mse": test_mse, "test_rmse": test_rmse})

In [11]:
def train_models_for_all_iot_devices(data: pd.DataFrame) -> None:
    for device_id in data.device_id.unique():
        iot_device_df = data[data.device_id == device_id]
        train_model_for_an_iot_device(iot_device_df)

In [12]:
train_models_for_all_iot_devices(df)

2022/12/16 09:48:24 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/tinyclues/opt/miniconda3/envs/mlflow/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


## Train a model each month

In [13]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

def train_model_for_an_iot_device(device_df: pd.DataFrame, month_name: str) -> None:
    """ Trains a model for a single device """
    device_id = int(device_df.iloc[0].device_id)
    
    with mlflow.start_run(nested=True, run_name=f"device {device_id}"):
        mlflow.log_params({"device_id": device_id, "month": month_name})
        
        
        mlflow.sklearn.autolog()

        # Split data
        X = device_df[["AT", "V", "AP", "RH"]]
        y = device_df["PE"]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
        mlflow.log_params(
            {"nb_sample": len(X), "nb_training_samples": len(X_train), "nb_testing_samples": len(X_test)})

        # Fit model
        model = RandomForestRegressor()
        model.fit(X_train, y_train)

        # Evaluate the model
        y_pred = model.predict(X_test)

        test_mae = mean_absolute_error(y_test, y_pred)
        test_mse = mean_squared_error(y_test, y_pred)
        test_rmse = mean_squared_error(y_test, y_pred, squared=False)
        mlflow.log_metrics({"test_mae": test_mae, "test_mse": test_mse, "test_rmse": test_rmse})

In [14]:
def train_models_for_all_iot_devices(data: pd.DataFrame, month_name: str) -> None:
    with mlflow.start_run(run_name=month_name):
        for device_id in data.device_id.unique():
            iot_device_df = data[data.device_id == device_id]
            train_model_for_an_iot_device(iot_device_df, month_name)

In [38]:
train_models_for_all_iot_devices(df, "june")

In [16]:
train_models_for_all_iot_devices(df, "july")

In [17]:
train_models_for_all_iot_devices(df, "august")

2022/12/16 09:50:22 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/qh/cbz8lgxn48g_00mb2gcbkcp40000gn/T/tmp7sp5h60l/model/model.pkl, flavor: sklearn), fall back to return ['scikit-learn==1.1.3', 'cloudpickle==2.2.0']. Set logging level to DEBUG to see the full traceback.


# 4. MLflow registry hands-on

## 4.1. Register MLflow Model in the MLflow registry

Documentation => https://mlflow.org/docs/latest/model-registry.html#adding-an-mlflow-model-to-the-model-registry

- Get all the runs for the current experiment from mlflow (runs from all months)

In [143]:
experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
all_experiment_runs_df = mlflow.search_runs(experiment_id)
all_experiment_runs_df.sample(5)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.test_rmse,metrics.training_r2_score,metrics.test_mae,metrics.training_mse,metrics.training_mae,metrics.mean_absolute_error_X_test,metrics.mean_squared_error_X_test,metrics.training_score,metrics.mean_squared_error-2_X_test,metrics.training_rmse,metrics.test_mse,params.max_samples,params.criterion,params.device_id,params.warm_start,params.n_jobs,params.bootstrap,params.oob_score,params.max_leaf_nodes,params.nb_sample,params.month,params.verbose,params.nb_training_samples,params.min_impurity_decrease,params.random_state,params.min_weight_fraction_leaf,params.min_samples_leaf,params.ccp_alpha,params.max_depth,params.n_estimators,params.max_features,params.nb_testing_samples,params.min_samples_split,tags.mlflow.runName,tags.estimator_class,tags.mlflow.log-model.history,tags.mlflow.user,tags.mlflow.source.name,tags.estimator_name,tags.mlflow.parentRunId,tags.mlflow.source.type
3,9be1ab4d8eb0472bb56fee952ff42066,1,FINISHED,/tmp/mlruns/1/9be1ab4d8eb0472bb56fee952ff42066...,2022-12-16 11:11:53.921000+00:00,2022-12-16 11:12:02.221000+00:00,2.233147,0.997256,1.345774,0.791433,0.535969,1.345774,4.986947,0.997256,2.233147,0.889625,4.986947,None,squared_error,3,False,None,True,False,None,11897,june,0,9517,0.0,None,0.0,1,0.0,None,100,1.0,2380,2,device 3,sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""9be1ab4d8eb0472bb56fee952ff42066""...",tinyclues,/Users/tinyclues/opt/miniconda3/envs/mlflow/li...,RandomForestRegressor,2754432b207a4654a4658e2a321c2653,LOCAL
22,596188bf19d447a794cca59b80e63e9c,1,FINISHED,/tmp/mlruns/1/596188bf19d447a794cca59b80e63e9c...,2022-12-16 08:48:25.132000+00:00,2022-12-16 08:48:32.271000+00:00,2.101418,0.997125,1.319873,0.836200,0.546136,NaN,NaN,0.997125,NaN,0.914440,4.415960,None,squared_error,None,False,None,True,False,None,11995,None,0,9596,0.0,None,0.0,1,0.0,None,100,1.0,2399,2,bald-panda-811,sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""596188bf19d447a794cca59b80e63e9c""...",tinyclues,/Users/tinyclues/opt/miniconda3/envs/mlflow/li...,RandomForestRegressor,None,LOCAL
18,065803583559488cbe392a404bbef287,1,FINISHED,/tmp/mlruns/1/065803583559488cbe392a404bbef287...,2022-12-16 08:48:46.739000+00:00,2022-12-16 08:48:53.637000+00:00,2.237516,0.997298,1.350027,0.779482,0.539185,1.350027,5.006476,0.997298,2.237516,0.882883,5.006476,None,squared_error,3,False,None,True,False,None,11897,june,0,9517,0.0,None,0.0,1,0.0,None,100,1.0,2380,2,device 3,sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""065803583559488cbe392a404bbef287""...",tinyclues,/Users/tinyclues/opt/miniconda3/envs/mlflow/li...,RandomForestRegressor,48a5fdf5dc7048ad96d480591f8a578f,LOCAL
11,267bd077cb394143a1f8af377609e5ae,1,FINISHED,/tmp/mlruns/1/267bd077cb394143a1f8af377609e5ae...,2022-12-16 08:49:30.230000+00:00,2022-12-16 08:49:39.208000+00:00,2.188679,0.997260,1.401955,0.807220,0.527504,1.401955,4.790314,0.997260,2.188679,0.898454,4.790314,None,squared_error,1,False,None,True,False,None,12081,july,0,9664,0.0,None,0.0,1,0.0,None,100,1.0,2417,2,device 1,sklearn.ensemble._forest.RandomForestRegressor,"[{""run_id"": ""267bd077cb394143a1f8af377609e5ae""...",tinyclues,/Users/tinyclues/opt/miniconda3/envs/mlflow/li...,RandomForestRegressor,c642630ba3414c8ca53e7d4a9a0e0a5f,LOCAL
19,48a5fdf5dc7048ad96d480591f8a578f,1,FINISHED,/tmp/mlruns/1/48a5fdf5dc7048ad96d480591f8a578f...,2022-12-16 08:48:46.720000+00:00,2022-12-16 08:49:15.300000+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,june,None,None,tinyclues,/Users/tinyclues/opt/miniconda3/envs/mlflow/li...,None,None,LOCAL


- Get the best run_id for the `device 1` in `june`: to do that we can use:
    - `filter_string` argument to filter on the device_id and month
    - `order_by` argument to order the results by our target metric (here test_rmse)

In [144]:
best_run_id = mlflow.search_runs(
    experiment_id,
    # filter on device_id 1 and month june
    filter_string=f"params.device_id = '1' AND params.month = 'june'",
    # order by the rmse
    order_by=["metrics.test_rmse asc"],
).iloc[0]["run_id"]
best_run_id

'3e760d7f3ceb40f9b690ec6a95bd1ef3'

- Create model_uri from the run_id

In [151]:
model_uri = f"runs:/{best_run_id}/model"
model_uri

'runs:/3e760d7f3ceb40f9b690ec6a95bd1ef3/model'

- Register model in the Registry

In [153]:
model_name_in_model_registry = f"powerplant_device_{device_id}"
model_version = mlflow.register_model(model_uri=model_uri, name=model_name_in_model_registry)
model_version

Registered model 'powerplant_device_2' already exists. Creating a new version of this model...
2022/12/16 17:51:25 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: powerplant_device_2, version 6
Created version '6' of model 'powerplant_device_2'.


<ModelVersion: creation_timestamp=1671209485365, current_stage='None', description='', last_updated_timestamp=1671209485365, name='powerplant_device_2', run_id='3e760d7f3ceb40f9b690ec6a95bd1ef3', run_link='', source='/tmp/mlruns/1/3e760d7f3ceb40f9b690ec6a95bd1ef3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='6'>

- Create a function for registring a model for a given device and month

In [154]:
def register_iot_device_model_with_best_rmse(month: str, device_id: int, experiment_name: str):
    experiment_id = mlflow.get_experiment_by_name(experiment_name).experiment_id
    run_id = mlflow.search_runs(
        experiment_id,
        order_by=['metrics.test_rmse asc'],
        filter_string=f"params.device_id = '{device_id}' AND params.month = '{month}'"
    ).iloc[0]["run_id"]
    
    model_name_in_model_registry = f"powerplant_device_{device_id}"
    model_version = mlflow.register_model(model_uri=f"runs:/{run_id}/model", name=model_name_in_model_registry)
    return model_version

In [164]:
current_month = "june"
for device_id in df.device_id.unique():
    _ = register_iot_device_model_with_best_rmse(
        month=current_month, device_id=device_id, experiment_name=experiment_name
    )

Registered model 'powerplant_device_3' already exists. Creating a new version of this model...
2022/12/16 18:03:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: powerplant_device_3, version 4
Created version '4' of model 'powerplant_device_3'.
Registered model 'powerplant_device_4' already exists. Creating a new version of this model...
2022/12/16 18:03:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: powerplant_device_4, version 4
Created version '4' of model 'powerplant_device_4'.
Registered model 'powerplant_device_1' already exists. Creating a new version of this model...
2022/12/16 18:03:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: powerplant_device_1, version 7
Created version '7' of model 'power

## 4.2. Transition an MLflow Model between different stages

Documentation => https://mlflow.org/docs/latest/model-registry.html#transitioning-an-mlflow-models-stage

In [156]:
client = MlflowClient()

In [56]:
help(client.transition_model_version_stage)

Help on method transition_model_version_stage in module mlflow.tracking.client:

transition_model_version_stage(name: str, version: str, stage: str, archive_existing_versions: bool = False) -> mlflow.entities.model_registry.model_version.ModelVersion method of mlflow.tracking.client.MlflowClient instance
    Update model version stage.
    
    :param name: Registered model name.
    :param version: Registered model version.
    :param stage: New desired stage for this model version.
    :param archive_existing_versions: If this flag is set to ``True``, all existing model
        versions in the stage will be automatically moved to the "archived" stage. Only valid
        when ``stage`` is ``"staging"`` or ``"production"`` otherwise an error will be raised.
    
    :return: A single :py:class:`mlflow.entities.model_registry.ModelVersion` object.
    
    .. code-block:: python
        :caption: Example
    
        import mlflow.sklearn
        from mlflow import MlflowClient
        

- Transition a model using the model_name and model_version_number

In [157]:
model_version = client.transition_model_version_stage(
    name="powerplant_device_1", version=4, stage="Production", archive_existing_versions=True)

In [158]:
model_version = client.transition_model_version_stage(
    name="powerplant_device_1", version=5, stage="Staging", archive_existing_versions=True)

In [159]:
model_version

<ModelVersion: creation_timestamp=1671190157448, current_stage='Staging', description='', last_updated_timestamp=1671209630176, name='powerplant_device_1', run_id='3e760d7f3ceb40f9b690ec6a95bd1ef3', run_link='', source='/tmp/mlruns/1/3e760d7f3ceb40f9b690ec6a95bd1ef3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>

- Transition a model using its model_version

In [160]:
print(f"Model name = {model_version.name}, model version = {model_version.version},"\
      f" stage = {model_version.current_stage}")

Model name = powerplant_device_1, model version = 5, stage = Staging


In [161]:
model_version = client.transition_model_version_stage(
    name=model_version.name, version=model_version.version, stage="archived")
print(f"Model name = {model_version.name}, model version = {model_version.version},"\
      f" stage = {model_version.current_stage}")

Model name = powerplant_device_1, model version = 5, stage = Archived


- Create a function that takes a model_version and a stage and transition the model to a given stage

In [162]:
def transition_model_to_a_new_stage(model_version, stage: str, archive_existing_versions: bool = True):
    valid_stage_values = ["staging", "production", "archived", "none"]
    assert stage.lower() in valid_stage_values, f"Invalid stage: {stage}. Valid stage values = {valid_stage_values}"
    client = MlflowClient()
    updated_model_version = client.transition_model_version_stage(
        name=model_version.name, version=model_version.version, stage=stage,
        archive_existing_versions=archive_existing_versions
    )
    return updated_model_version

In [171]:
updated_model_version = transition_model_to_a_new_stage(model_version, "Staging")
print(f"Model name = {model_version.name}, model version = {model_version.version},"\
      f" stage = {model_version.current_stage}")

Model name = powerplant_device_1, model version = 5, stage = Archived


## 4.3. Fetch an MLflow Model from the Model Registry

- Get all the latest version for a given model in all the stages

In [172]:
latest_model_version_list = client.get_latest_versions(name="powerplant_device_1")
latest_model_version_list

[<ModelVersion: creation_timestamp=1671210203744, current_stage='None', description='', last_updated_timestamp=1671210203744, name='powerplant_device_1', run_id='3e760d7f3ceb40f9b690ec6a95bd1ef3', run_link='', source='/tmp/mlruns/1/3e760d7f3ceb40f9b690ec6a95bd1ef3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='7'>,
 <ModelVersion: creation_timestamp=1671190060068, current_stage='Archived', description='', last_updated_timestamp=1671210506574, name='powerplant_device_1', run_id='3e760d7f3ceb40f9b690ec6a95bd1ef3', run_link='', source='/tmp/mlruns/1/3e760d7f3ceb40f9b690ec6a95bd1ef3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='4'>,
 <ModelVersion: creation_timestamp=1671190157448, current_stage='Production', description='', last_updated_timestamp=1671210506574, name='powerplant_device_1', run_id='3e760d7f3ceb40f9b690ec6a95bd1ef3', run_link='', source='/tmp/mlruns/1/3e760d7f3ceb40f9b690ec6a95bd1ef3/artifacts/model', s

In [173]:
for latest_model_version in latest_model_version_list:
    model_version = latest_model_version.version
    model_stage = latest_model_version.current_stage
    print(f"Model version {model_version} is the latest version in the stage * {model_stage} *")

Model version 7 is the latest version in the stage * None *
Model version 4 is the latest version in the stage * Archived *
Model version 5 is the latest version in the stage * Production *


### Use case 1: get the model in Production and make predictions with

- Get the models in the Production stage

In [175]:
production_model_version = client.get_latest_versions(name="powerplant_device_1", stages=["Production"])[0]
production_model_version

<ModelVersion: creation_timestamp=1671190157448, current_stage='Production', description='', last_updated_timestamp=1671210506574, name='powerplant_device_1', run_id='3e760d7f3ceb40f9b690ec6a95bd1ef3', run_link='', source='/tmp/mlruns/1/3e760d7f3ceb40f9b690ec6a95bd1ef3/artifacts/model', status='READY', status_message='', tags={}, user_id='', version='5'>

- Get model uri in the MLflow Registry

In [177]:
model_uri = f"models:/{production_model_version.name}/{production_model_version.version}"

- Load the model

In [178]:
model = mlflow.pyfunc.load_model(model_uri=model_uri)

- Make predictions with the model

In [179]:
inference_df = df[df['device_id'] == 1][['AT', 'V', 'AP', 'RH']].sample(5)
model.predict(inference_df)

array([448.6766, 431.599 , 494.2972, 448.1004, 467.1803])

### Use case 2 (shadow_production): get the models in Production and Staging and make predictions with

In [110]:
# TODO

# 5. Training pipeline

In this step, we will be creating a training pipeline:
1. Train the candidate model on the new data
2. Evaluate the candidate model on the ground truth dataset
3. Get the baseline model from the ModelRegistry and evaluate it on the ground truth dataset
4. Compare the candidate and baseline metrics, if they are better, register the candidate model in the ModelRegistry and transition it to Production

Bonus:
- check if we have enough data for training before launching the pipeline

## 5.1. Simulate data drift

In [111]:
# TODO

## 5.2. Create the training pipeline

In [112]:
# TODO